<div class="alert alert-info">

## Introduction


</div>

In [22]:
#import
import os

import numpy as np
import pandas as pd
from hashlib import sha1

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import accuracy

<div class="alert alert-info">

## Data Description<a name="2"></a>
Given the large size of the dataset, only 10000 rows of the dataset is used for the models.
This project utilizes a comprehensive dataset sourced from Kaggle, which can be accessed via the following link: (https://www.kaggle.com/datasets/beaglelee/amazon-reviews-us-books-v1-02-tsv-zip). The dataset consists of 15 columns and encompasses a substantial total of 3,105,370 rows, providing rich insights into customer feedback and product ratings specifically within the book category.

Due to the extensive size of the dataset, a subset of 10,000 rows has been selected for analysis and modeling. This reduction allows for efficient processing while still capturing the diverse range of reviews and ratings present in the original dataset.
</div>

In [23]:
# Data
data = pd.read_csv("data/amazon_reviews_us_Books_v1_02.tsv", sep='\t', on_bad_lines='skip')


In [24]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12076615,RQ58W7SMO911M,0385730586,122662979,Sisterhood of the Traveling Pants (Book 1),Books,4.0,2.0,3.0,N,N,this book was a great learning novel!,this boook was a great one that you could lear...,2005-10-14
1,US,12703090,RF6IUKMGL8SF,0811828964,56191234,The Bad Girl's Guide to Getting What You Want,Books,3.0,5.0,5.0,N,N,Fun Fluff,If you are looking for something to stimulate ...,2005-10-14
2,US,12257412,R1DOSHH6AI622S,1844161560,253182049,"Eisenhorn (A Warhammer 40,000 Omnibus)",Books,4.0,1.0,22.0,N,N,this isn't a review,never read it-a young relative idicated he lik...,2005-10-14
3,US,50732546,RATOTLA3OF70O,0373836635,348672532,Colby Conspiracy (Colby Agency),Books,5.0,2.0,2.0,N,N,fine author on her A-game,Though she is honored to be Chicago Woman of t...,2005-10-14
4,US,51964897,R1TNWRKIVHVYOV,0262181533,598678717,The Psychology of Proof: Deductive Reasoning i...,Books,4.0,0.0,2.0,N,N,Execellent cursor examination,Review based on a cursory examination by Unive...,2005-10-14


<div class="alert alert-info">

## Exploratory Data Analysis(EDA) <a name="3"></a>

This section outlines the exploratory data analysis techniques employed to extract meaningful insights from the dataset, guiding the subsequent steps in model development.

To create a focused subset for analysis, we identified the 1,000 unique product IDs with the highest number of reviews. From this selection, we sampled a total of 10,000 rows corresponding to their reviews. As a result, our final subset comprises 10,000 rows, encompassing 993 distinct products and 9,993 associated reviews. This approach ensures a diverse representation of products while allowing us to conduct a thorough examination of customer feedback.

</div>

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105370 entries, 0 to 3105369
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   marketplace        object 
 1   customer_id        int64  
 2   review_id          object 
 3   product_id         object 
 4   product_parent     int64  
 5   product_title      object 
 6   product_category   object 
 7   star_rating        float64
 8   helpful_votes      float64
 9   total_votes        float64
 10  vine               object 
 11  verified_purchase  object 
 12  review_headline    object 
 13  review_body        object 
 14  review_date        object 
dtypes: float64(3), int64(2), object(10)
memory usage: 355.4+ MB


In [26]:
data.isnull().sum()

marketplace            0
customer_id            0
review_id              0
product_id             0
product_parent         0
product_title          0
product_category       0
star_rating            4
helpful_votes          4
total_votes            4
vine                   4
verified_purchase      4
review_headline       57
review_body            4
review_date          133
dtype: int64

In [27]:
data = data.dropna()

In [28]:
data.replace(['null', 'N/A', '', ' '], np.nan, inplace=True)

In [29]:
data.isnull().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64

In [30]:
data.nunique()

marketplace                1
customer_id          1502265
review_id            3105184
product_id            779692
product_parent        666003
product_title         713665
product_category           1
star_rating                5
helpful_votes            942
total_votes             1024
vine                       2
verified_purchase          2
review_headline      2456998
review_body          3070458
review_date             3575
dtype: int64

In [31]:
# Selected a subset with customers and products with at least 10 reviews
# Step 1: Filter customers with at least 10 reviews
customer_review_counts = data.groupby('customer_id').size().reset_index(name='review_count')
customers_with_at_least_10_reviews = customer_review_counts[customer_review_counts['review_count'] >= 10]

# Step 2: Filter products with at least 10 reviews
product_review_counts = data.groupby('product_id').size().reset_index(name='review_count')
products_with_at_least_10_reviews = product_review_counts[product_review_counts['review_count'] >= 10]

# Step 3: Filter the original dataset to only include customers and products with at least 10 reviews
filtered_data = data[
    (data['customer_id'].isin(customers_with_at_least_10_reviews['customer_id'])) &
    (data['product_id'].isin(products_with_at_least_10_reviews['product_id']))
]
filtered_data.shape

(469085, 15)

In [32]:

customer_review_counts_filtered = filtered_data.groupby('customer_id').size().reset_index(name='review_count')
product_review_counts_filtered = filtered_data.groupby('product_id').size().reset_index(name='review_count')

customers_with_at_least_10_reviews = customer_review_counts_filtered[customer_review_counts_filtered['review_count'] >= 10]
products_with_at_least_10_reviews = product_review_counts_filtered[product_review_counts_filtered['review_count'] >= 10]


num_customers_with_at_least_10_reviews = len(customers_with_at_least_10_reviews)
num_products_with_at_least_10_reviews = len(products_with_at_least_10_reviews)

print(f"Number of customers with at least 10 reviews: {num_customers_with_at_least_10_reviews}")
print(f"Number of products with at least 10 reviews: {num_products_with_at_least_10_reviews}")

Number of customers with at least 10 reviews: 14364
Number of products with at least 10 reviews: 13511


In [33]:
# Step 1: Filter customers with at least 10 reviews
customer_review_counts = filtered_data.groupby('customer_id').size().reset_index(name='review_count')
customers_with_at_least_10_reviews = customer_review_counts[customer_review_counts['review_count'] >= 10]['customer_id']

# Step 2: Filter products with at least 10 reviews
product_review_counts = filtered_data.groupby('product_id').size().reset_index(name='review_count')
products_with_at_least_10_reviews = product_review_counts[product_review_counts['review_count'] >= 10]['product_id']

# Step 3: Filter the dataset to include only reviews from these customers and products
filtered_customers_products_data = filtered_data[
    (filtered_data['customer_id'].isin(customers_with_at_least_10_reviews)) & 
    (filtered_data['product_id'].isin(products_with_at_least_10_reviews))
]

# Step 4: Select a subset of customers (e.g., 1500) and products (e.g., 2000)
subset_customers = customers_with_at_least_10_reviews.sample(n=1500, random_state=42)
subset_products = products_with_at_least_10_reviews.sample(n=2000, random_state=42)

# Step 5: Filter the dataset to include only reviews from the selected customers and products
subset_data = filtered_customers_products_data[
    (filtered_customers_products_data['customer_id'].isin(subset_customers)) & 
    (filtered_customers_products_data['product_id'].isin(subset_products))
]

# Step 6: Check the number of available reviews before sampling
available_reviews = len(subset_data)
print(f"Available reviews: {available_reviews}")

# If there are fewer than 10,000 reviews, sample only as many as are available
if available_reviews < 10000:
    final_sample = subset_data.sample(n=available_reviews, random_state=123)
else:
    final_sample = subset_data.sample(n=10000, random_state=123)

# Step 7: Display the results
final_num_reviews = len(final_sample)
final_num_customers = final_sample['customer_id'].nunique()
final_num_products = final_sample['product_id'].nunique()

print(f"Final number of reviews: {final_num_reviews}")
print(f"Final number of unique customers: {final_num_customers}")
print(f"Final number of unique products: {final_num_products}")


Available reviews: 3761
Final number of reviews: 3761
Final number of unique customers: 1162
Final number of unique products: 1500


In [34]:
final_sample.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
2074179,US,50913245,R29BRD85IOWKVT,0553110721,698766211,Dune: House Harkonnen,Books,4.0,2.0,3.0,N,N,More of the back stories on &quot;Dune&quot;'s...,Not only is it true that if you have not read ...,2001-03-10
1392079,US,41064927,R33O8N15HPKDLI,0553111248,417245650,Passage,Books,5.0,1.0,2.0,N,N,A Lyrical SF Mystery,"A lyrical, intricately-plotted science fiction...",2002-10-01
646946,US,52562548,R27CCQBEUWIAB9,0316000647,657855247,Absolute Friends,Books,5.0,1.0,5.0,N,N,Involving Thriller with Arch Style,"I enjoyed this tale of intrigue, with the prot...",2004-06-01
1287132,US,52290154,RU2OQRH3YFW46,0375422412,120680919,Three Junes,Books,2.0,6.0,12.0,N,N,Disappointing,I have a friend who liked this novel so when i...,2002-12-31
2949131,US,51376733,R28NIFWZUG9907,0060191988,665059037,A Night Without Armor: Poems,Books,1.0,0.0,1.0,N,N,Jewel: Queen of the hollow verse,I've read the book several times and while ana...,1998-12-17


In [35]:
final_sample.nunique()

marketplace             1
customer_id          1162
review_id            3761
product_id           1500
product_parent       1476
product_title        1487
product_category        1
star_rating             5
helpful_votes         152
total_votes           165
vine                    1
verified_purchase       2
review_headline      3648
review_body          3746
review_date          1798
dtype: int64

In [36]:
final_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3761 entries, 2074179 to 2774649
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   marketplace        3761 non-null   object 
 1   customer_id        3761 non-null   int64  
 2   review_id          3761 non-null   object 
 3   product_id         3761 non-null   object 
 4   product_parent     3761 non-null   int64  
 5   product_title      3761 non-null   object 
 6   product_category   3761 non-null   object 
 7   star_rating        3761 non-null   float64
 8   helpful_votes      3761 non-null   float64
 9   total_votes        3761 non-null   float64
 10  vine               3761 non-null   object 
 11  verified_purchase  3761 non-null   object 
 12  review_headline    3761 non-null   object 
 13  review_body        3761 non-null   object 
 14  review_date        3761 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 470.1+ KB


In [37]:
final_sample.to_csv('data/amazon_reviews_subset.csv', index=False)


<div class="alert alert-info">
    
## Collaborative Filtering
**Collaborative Filtering** is a widely-used technique for addressing the challenge of missing entries in a utility matrix, leveraging user behavior and interactions to make recommendations. This approach operates on the principle that users who have agreed in the past will continue to agree in the future, allowing the model to infer preferences based on the preferences of similar users.

This method can be likened to advanced dimensionality reduction techniques such as Latent Semantic Analysis (LSA) or Truncated Singular Value Decomposition (SVD). By capturing the underlying relationships between users and items, collaborative filtering helps to predict missing values, enhancing the accuracy and relevance of recommendations.

In this project, we will implement collaborative filtering as our baseline model to improve user experience by personalizing content based on historical data, thus enabling more informed decision-making.
</div>

In [53]:
# Reading the data
coll_data = final_sample[['customer_id', 'product_id', 'star_rating']].reset_index(drop=True)
coll_data.head()

,customer_id,product_id,star_rating
0,50913245,0553110721,4.0
1,41064927,0553111248,5.0
2,52562548,0316000647,5.0
3,52290154,0375422412,2.0
4,51376733,0060191988,1.0


In [54]:
# Number of customers and products
user_key = "customer_id"
item_key = "product_id"
N = len(np.unique(coll_data[user_key])) 
M = len(np.unique(coll_data[item_key]))
print(f"Number of customers (N)  : {N}")
print(f"Number of products (M) : {M}")

Number of customers (N)  : 1162
Number of products (M) : 1500


In [55]:
non_nan_ratings_percentage = (len(coll_data) / (N * M) * 100) 
print(f"Non-nan ratings percentage: {np.round(non_nan_ratings_percentage,3)}")

Non-nan ratings percentage: 0.216


In [56]:
avg_nratings_per_user = coll_data.groupby(user_key).size().mean()
avg_nratings_per_movie = coll_data.groupby(item_key).size().mean()
print(f"Average number of ratings per customer : {avg_nratings_per_user}")
print(f"Average number of ratings per product: {avg_nratings_per_movie}")

Average number of ratings per customer : 3.2366609294320137
Average number of ratings per product: 2.5073333333333334


In [57]:
# Data Splitting
X = coll_data.copy()
y = coll_data['customer_id']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=123)

In [58]:
user_mapper = dict(zip(np.unique(coll_data[user_key]), list(range(N))))
item_mapper = dict(zip(np.unique(coll_data[item_key]), list(range(M))))
user_inverse_mapper = dict(zip(list(range(N)), np.unique(coll_data[user_key])))
item_inverse_mapper = dict(zip(list(range(M)), np.unique(coll_data[item_key])))

In [60]:
def create_Y_from_ratings(data, N, M):
    Y = np.zeros((N, M))
    Y.fill(np.nan)
    for index, val in data.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        Y[n, m] = val["star_rating"]

    return Y

train_mat = create_Y_from_ratings(X_train, N, M)
valid_mat = create_Y_from_ratings(X_valid, N, M)


In [61]:
# What's the number of non-nan elements in train_mat (nnn_train_mat)?
nnn_train_mat = np.sum(~np.isnan(train_mat)) 

# What's the number of non-nan elements in valid_mat (nnn_valid_mat)?
nnn_valid_mat = np.sum(~np.isnan(valid_mat)) 
print(f"Number of non-nan elements in train_mat: {nnn_train_mat}")
print(f"Number of non-nan elements in valid_mat: {nnn_valid_mat}")

Number of non-nan elements in train_mat: 2947
Number of non-nan elements in valid_mat: 748


In [62]:
# Evaluation
def error(Y1, Y2):
    """
    Given two matrices of the same shape, 
    returns the root mean squared error (RMSE).
    """
    return np.sqrt(np.nanmean((Y1 - Y2) ** 2))


def evaluate(pred_Y, train_mat, valid_mat, model_name="Global average"):
    """
    Given predicted utility matrix and train and validation utility matrices 
    print train and validation RMSEs.
    """
    print("%s train RMSE: %0.2f" % (model_name, error(pred_Y, train_mat)))
    print("%s valid RMSE: %0.2f" % (model_name, error(pred_Y, valid_mat)))

In [63]:
# global average rating baseline
avg = np.nanmean(train_mat)
pred_g = np.zeros(train_mat.shape) + avg
evaluate(pred_g, train_mat, valid_mat, model_name="Global average")

Global average train RMSE: 1.11
Global average valid RMSE: 1.12


In [64]:
# Per-user average baseline
avg_n = np.nanmean(train_mat, axis=1)
avg_n[
    np.isnan(avg_n)
] = avg  
pred_n = np.tile(avg_n[:, None], (1, M))
evaluate(pred_n, train_mat, valid_mat, model_name="Per-user average")

Per-user average train RMSE: 0.77
Per-user average valid RMSE: 1.18


/var/folders/j6/44dd1c8s33xg2kn35nxq3f5m0000gn/T/ipykernel_48949/2070846812.py:1: RuntimeWarning: Mean of empty slice
  avg_n = np.nanmean(train_mat, axis=1)


In [65]:
# Per-product average baseline
avg_m = np.nanmean(train_mat, axis=0)
avg_m[np.isnan(avg_m)] = avg
pred_m = np.tile(avg_m[None, :], (N, 1))
evaluate(pred_m, train_mat, valid_mat, model_name="Per-product average")

Per-product average train RMSE: 0.75
Per-product average valid RMSE: 1.28


/var/folders/j6/44dd1c8s33xg2kn35nxq3f5m0000gn/T/ipykernel_48949/1388612677.py:2: RuntimeWarning: Mean of empty slice
  avg_m = np.nanmean(train_mat, axis=0)


In [66]:
# Average of per-user and per-product average baselines
pred_n_m = (pred_n + pred_m) * 0.5
evaluate(pred_n_m, train_mat, valid_mat, model_name="Per-user and product average")

Per-user and product average train RMSE: 0.65
Per-user and product average valid RMSE: 1.11


In [67]:
# K-nearest neighbours imputation
from sklearn.impute import KNNImputer

num_neighs = [10, 15, 18, 20, 40]
for n_neighbors in num_neighs:
    print("\nNumber of neighbours: ", n_neighbors)
    imputer = KNNImputer(n_neighbors=n_neighbors, keep_empty_features=True)
    pred_knn = imputer.fit_transform(train_mat)
    evaluate(pred_knn, train_mat, valid_mat)


Number of neighbours:  10
Global average train RMSE: 0.00
Global average valid RMSE: 2.27

Number of neighbours:  15
Global average train RMSE: 0.00
Global average valid RMSE: 2.27

Number of neighbours:  18
Global average train RMSE: 0.00
Global average valid RMSE: 2.27

Number of neighbours:  20
Global average train RMSE: 0.00
Global average valid RMSE: 2.27

Number of neighbours:  40
Global average train RMSE: 0.00
Global average valid RMSE: 2.27


In [68]:
# collaborative filtering with TruncatedSVD()
def reconstruct_svd(Z, W, avg_n, avg_m):
    return Z @ W + 0.5 * avg_n[:, None] + 0.5 * avg_m[None]


train_mat_svd = train_mat - 0.5 * avg_n[:, None] - 0.5 * avg_m[None]
train_mat_svd = np.nan_to_num(train_mat_svd)

k_range = [3, 5, 10, 20, 30, 200, 400, 1000]
for k in k_range:
    print("\n")
    tsvd = TruncatedSVD(n_components=k)
    Z = tsvd.fit_transform(train_mat_svd)
    W = tsvd.components_
    X_hat = reconstruct_svd(Z, W, avg_n, avg_m)
    evaluate(X_hat, train_mat, valid_mat, model_name="TruncatedSVD (k = %d)" % k)



TruncatedSVD (k = 3) train RMSE: 0.63
TruncatedSVD (k = 3) valid RMSE: 1.11


TruncatedSVD (k = 5) train RMSE: 0.61
TruncatedSVD (k = 5) valid RMSE: 1.11


TruncatedSVD (k = 10) train RMSE: 0.58
TruncatedSVD (k = 10) valid RMSE: 1.11


TruncatedSVD (k = 20) train RMSE: 0.54
TruncatedSVD (k = 20) valid RMSE: 1.11


TruncatedSVD (k = 30) train RMSE: 0.50
TruncatedSVD (k = 30) valid RMSE: 1.11


TruncatedSVD (k = 200) train RMSE: 0.19
TruncatedSVD (k = 200) valid RMSE: 1.11


TruncatedSVD (k = 400) train RMSE: 0.07
TruncatedSVD (k = 400) valid RMSE: 1.11


TruncatedSVD (k = 1000) train RMSE: 0.00
TruncatedSVD (k = 1000) valid RMSE: 1.11


In [69]:
# Using surprise package
reader = Reader()
data = Dataset.load_from_df(coll_data, reader)  

k = 10
algo = SVD(n_factors=k, random_state=42)

In [70]:
pd.DataFrame(cross_validate(algo, data, measures=["RMSE"], cv=5, verbose=True))

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0809  1.0490  1.0400  1.0465  1.0327  1.0498  0.0165  
Fit time          0.01    0.00    0.00    0.01    0.00    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


,test_rmse,fit_time,test_time
0,1.080948,0.011908,0.001599
1,1.049049,0.004886,0.001480
2,1.040027,0.004930,0.001348
3,1.046483,0.005005,0.001360
4,1.032738,0.004679,0.001345
